<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">


# Parallel and Distributed Machine Learning

[Dask-ML](https://dask-ml.readthedocs.io) has resources for parallel and distributed machine learning.


## Types of Scaling

There are a couple of distinct scaling problems you might face.
The scaling strategy depends on which problem you're facing.

1. CPU-Bound: Data fits in RAM, but training takes too long. Many hyperparameter combinations, a large ensemble of many models, etc.
2. Memory-bound: Data is larger than RAM, and sampling isn't an option.

* For in-memory problems, just use scikit-learn (or your favorite ML library).
* For large models, use `dask_ml.joblib` and your favorite scikit-learn estimator
* For large datasets, use `dask_ml` estimators

![](images/ml-dimensions.png)

## Scikit-Learn in 5 Minutes

Scikit-Learn has a nice, consistent API.

1. You instantiate an `Estimator` (e.g. `LinearRegression`, `RandomForestClassifier`, etc.). All of the models *hyperparameters* (user-specified parameters, not the ones learned by the estimator) are passed to the estimator when it's created.
2. You call `estimator.fit(X, y)` to train the estimator.
3. Use `estimator` to inspect attributes, make predictions, etc. 

Let's generate some random data.

In [1]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10000, n_features=4, random_state=0)
X[:8]

array([[-0.77244139,  0.3607576 , -2.38110133,  0.08757   ],
       [ 1.14946035,  0.62254594,  0.37302939,  0.45965795],
       [-1.90879217, -1.1602627 , -0.27364545, -0.82766028],
       [-0.77694695,  0.31434299, -2.26231851,  0.06339125],
       [-1.17047054,  0.02212382, -2.17376797, -0.13421976],
       [ 0.79010037,  0.68530624, -0.44740487,  0.44692959],
       [ 1.68616989,  1.6329131 , -1.42072654,  1.04050557],
       [-0.93912893, -1.02270838,  1.10093827, -0.63714432]])

In [2]:
y[:8]

array([0, 0, 1, 0, 0, 0, 0, 1])

We'll fit a Support Vector Classifier.

In [3]:
from sklearn.svm import SVC

Create the estimator and fit it.

In [4]:
estimator = SVC(random_state=0)
estimator.fit(X, y)

SVC(random_state=0)

Inspect the learned attributes.

In [5]:
estimator.support_vectors_[:4]

array([[-0.77244139,  0.3607576 , -2.38110133,  0.08757   ],
       [ 1.14946035,  0.62254594,  0.37302939,  0.45965795],
       [-0.77694695,  0.31434299, -2.26231851,  0.06339125],
       [ 0.79010037,  0.68530624, -0.44740487,  0.44692959]])

Check the accuracy.

In [6]:
estimator.score(X, y)

0.905

## Hyperparameters

Most models have *hyperparameters*. They affect the fit, but are specified up front instead of learned during training.

In [7]:
estimator = SVC(C=0.00001, shrinking=False, random_state=0)
estimator.fit(X, y)
estimator.support_vectors_[:4]

array([[-0.77244139,  0.3607576 , -2.38110133,  0.08757   ],
       [ 1.14946035,  0.62254594,  0.37302939,  0.45965795],
       [-0.77694695,  0.31434299, -2.26231851,  0.06339125],
       [-1.17047054,  0.02212382, -2.17376797, -0.13421976]])

In [8]:
estimator.score(X, y)

0.5007

## Hyperparameter Optimization

There are a few ways to learn the best *hyper*parameters while training. One is `GridSearchCV`.
As the name implies, this does a brute-force search over a grid of hyperparameter combinations.

In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
%%time
estimator = SVC(gamma='auto', random_state=0, probability=True)
param_grid = {
    'C': [0.001, 10.0],
    'kernel': ['rbf', 'poly'],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2)
grid_search.fit(X, y)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] END ................................C=0.001, kernel=rbf; total time=   4.9s
[CV] END ................................C=0.001, kernel=rbf; total time=   5.1s
[CV] END ...............................C=0.001, kernel=poly; total time=   2.8s
[CV] END ...............................C=0.001, kernel=poly; total time=   3.1s
[CV] END .................................C=10.0, kernel=rbf; total time=   1.4s
[CV] END .................................C=10.0, kernel=rbf; total time=   1.4s
[CV] END ................................C=10.0, kernel=poly; total time=   2.6s
[CV] END ................................C=10.0, kernel=poly; total time=   2.4s
CPU times: user 26.8 s, sys: 953 ms, total: 27.7 s
Wall time: 27.7 s


GridSearchCV(cv=2,
             estimator=SVC(gamma='auto', probability=True, random_state=0),
             param_grid={'C': [0.001, 10.0], 'kernel': ['rbf', 'poly']},
             verbose=2)

## Single-machine parallelism with scikit-learn

![](images/unmerged_grid_search_graph.png)

Scikit-Learn has nice *single-machine* parallelism, via Joblib.
Any scikit-learn estimator that can operate in parallel exposes an `n_jobs` keyword.
This controls the number of CPU cores that will be used.

In [11]:
%%time
grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2, n_jobs=-1)
grid_search.fit(X, y)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
CPU times: user 3.96 s, sys: 259 ms, total: 4.22 s
Wall time: 14.2 s


GridSearchCV(cv=2,
             estimator=SVC(gamma='auto', probability=True, random_state=0),
             n_jobs=-1,
             param_grid={'C': [0.001, 10.0], 'kernel': ['rbf', 'poly']},
             verbose=2)

## Multi-machine parallelism with Dask

![](images/merged_grid_search_graph.png)

Dask can talk to scikit-learn (via joblib) so that your *cluster* is used to train a model. 

If you run this on a laptop, it will take quite some time, but the CPU usage will be satisfyingly near 100% for the duration. To run faster, you would need a distributed cluster. That would mean putting something in the call to `Client` something like

```
c = Client('tcp://my.scheduler.address:8786')
```

Details on the many ways to create a cluster can be found [here](https://docs.dask.org/en/latest/setup/single-distributed.html).

Let's try it on a larger problem (more hyperparameters).

In [12]:
import joblib
import dask.distributed

c = dask.distributed.Client()

In [13]:
param_grid = {
    'C': [0.001, 10.0],
    'kernel': ['rbf', 'poly'],
}

In [14]:
%%time
grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=5, n_jobs=-1)
with joblib.parallel_backend("dask", scatter=[X, y]):
    grid_search.fit(X, y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
CPU times: user 11.9 s, sys: 1.03 s, total: 12.9 s
Wall time: 43.2 s


In [15]:
grid_search.best_params_, grid_search.best_score_

({'C': 10.0, 'kernel': 'rbf'}, 0.9119000000000002)